Part B: Create a notebook called linear_regression<br>
<br>
Python: Initial data prep section.  Read, clean and create sets.<br>
Markdown: Pick an initial set of features for X and the target feature y.  Explain <br>why you made this choice.<br>
Python: Do a linear regression on X and y.  Compute metrics.<br>
Markdown: Comment on the results.<br>
Python: See if you can do better.<br>
Python: Do a final evaluation with the test set.<br>
Markdown: Look at the parameters you found and discuss what you have learned.<br>

In [2]:
import pandas as pd
#import the csv files
weather = pd.read_csv("seattle-weather.csv")

weather["date"] = pd.to_datetime(weather["date"])

#convert weather field into numerical values
weather_mapping = {'drizzle': 0, 'rain': 1, 'sun': 2, 'snow': 3, 'fog': 4}
weather["weather"] = weather["weather"].map(weather_mapping)

weather.dropna()
weather.drop(columns=["date"])

,precipitation,temp_max,temp_min,wind,weather
0,0.0,12.8,5.0,4.7,0
1,10.9,10.6,2.8,4.5,1
2,0.8,11.7,7.2,2.3,1
3,20.3,12.2,5.6,4.7,1
4,1.3,8.9,2.8,6.1,1
...,...,...,...,...,...
1456,8.6,4.4,1.7,2.9,1
1457,1.5,5.0,1.7,1.3,1
1458,0.0,7.2,0.6,2.6,4
1459,0.0,5.6,-1.0,3.4,2


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
train_set, test_set = train_test_split(weather,test_size=0.2,random_state=123)
reg = ElasticNet(alpha = 1.0, l1_ratio=0.5)

use = ["temp_max","wind", "precipitation"]
find = ["weather"]

I chose to train the data set using <br>
"precipitation", "temp_max", and, "wind"<br>
I chose to use these features because they showed a small correlation in the <br>
initial exploration notebook. I tryed it with temp_min included but the mse was similar so i chose to keep it out to help avoid overfitting. 

In [4]:
working_set = train_set.copy()

X_test = working_set[use]
Y_test = working_set[find]

reg.fit(X_test, Y_test)

Y_pred = reg.predict(test_set[use])

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(test_set[find], Y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.7110632507789894


The model had a Mean Squared Error value of 0.71. This is not that great  however with the data I have and what I am trying to predict, this is to be expected due to the predicted feature not being a actually value.  